In [1]:
import descarteslabs as dl
from datetime import datetime
from subprocess import check_output
import subprocess 
# from sys import argv
import json
import pickle

## Open File

In [ ]:
process = subprocess.call('mkdir temp', shell=True)

In [ ]:
args = ['gsutil cp gs://dl-fires/inciweb-data/conus_fires_20180517-18UTC.txt temp/']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print p.communicate()

In [2]:
def iterate_over_file(string):
    counter = 0
    for line in iter(string.splitlines()):
        if counter == 0:
            counter +=1
        else: 
            initiate_fire_instance(line)
            counter +=1

In [3]:
# This could be refactored to include a class 
def initiate_fire_instance(fire_string): 
    fire_arr = fire_string.split()
    name_list = fire_arr[:-4]
    name = " ".join(name_list)
    fire_id = fire_arr[-4]
    lat = float(fire_arr[-3])
    lon = float(fire_arr[-2])
    coords=[lat,lon]
    acres = fire_arr[-1]
    check_fire_status(fire_id, name, coords, acres)

In [4]:
txt = open('temp/conus_fires_20180517-18UTC.txt')
file_string_contents = txt.read()

In [13]:
# CURRENT RUN SCRIPT, though depends on the above being run first 
iterate_over_file(file_string_contents)

34.81
-101.31
2018-05-15
[
  [
    [-101.5019743784295, 34.643886689695925],
    [-101.10880271648934, 34.65132359977969],
    [-101.1171020650867, 34.97628075578136],
    [-101.51181743579816, 34.968753856917836],
    ...
  ]
]
'The number of total images found is 0.'
31.98
-109.37
2018-05-15
[
  [
    [-109.5634625151115, 31.819611165173043],
    [-109.1826033558315, 31.814741312370256],
    [-109.17618166476376, 32.13990428462667],
    [-109.5583852884468, 32.144835716341106],
    ...
  ]
]
'The number of total images found is 2.'
landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_035038_20180517_20180518_01_RT_v1
sentinel-2:L1C:2018-05-15_12SXA_52_S2B_v1
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS


### Check if fire instance exists 

In [5]:
def check_fire_status(fire_id, name, coordinates, acres):
    fires = get_fire_instances() 
    if fire_id in fires:
        last_checked =  fires[fire_id]['last_checked']
        existing_fire(name, coordinates, last_checked)
    else:
        print "PASS"
#     for fire in data:
#         print fire in data
#        

## Read Fire Directories 
Create a new one for the output if it does not exist 

In [6]:
args = ['gsutil ls gs://dl-fires/imagery']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
buckets= p.communicate()[0]
buckets = buckets.split('\ngs://dl-fires/imagery/')
buckets = [bucket.replace('/', '') for bucket in buckets]
buckets = [bucket.replace('\n', '') for bucket in buckets]
buckets = buckets[1:]
buckets

['test_fire', 'test_fire_2']

### Read in fire instances

In [7]:
def get_fire_instances(): 
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
    return b

## Logic for existing fire instance or new fire instance

In [ ]:
with open('/path/to/my_file.json', 'w') as f:
    data['new_key'] = [1, 2, 3]
    json.dump(data, f)

In [ ]:
def new_fire_instance(fire_id, name, coordinates, acres): 
#     find imagery 
#  write file 
    with open('temp/fires.json', 'w') as f:
        data[id] = [1, 2, 3]
        json.dump(data, f)

In [8]:
def existing_fire(name, coordinates, last_checked):
    find_imagery(name, last_checked ,coordinates) 
#     with open('temp/fires.json', 'rb') as handle:
#         data = pickle.load(handle)
#         data[fire_id]['last_checked'] = datetime.today().isoformat()
#     # Update fire instances' status
#     print(data)
#     with open('temp/fires.json', 'w') as f: 
#         json.dump(data, f)

## Find imagery

In [9]:
def make_tile(coords): 
    # to get the tile centered around your lat/lon, give it a tilesize of 1 and a pad of the half-width of your desired image (in meters)
    print coords[0]
    print coords[1]
    bounding_box = dl.raster.dltile_from_latlon(
        lat=coords[0],
        lon=coords[1], 
        resolution=30, 
        tilesize=2, 
        pad=600)

    return bounding_box
    

In [10]:
def search_products(date, geometry):
    print(date)
    from pprint import pprint 
    pprint(geometry['geometry']['coordinates'])
    images = dl.metadata.search(
        ['landsat:LC08:01:RT:TOAR', 'sentinel-2:L1C'],
        geom= geometry, 
        start_datetime=date, 
        end_datetime= datetime.today().isoformat()
        )
    pprint("The number of total images found is {}.".format(len(images['features'])))
    return [image['id'] for image in images['features']]

In [11]:
def download_imagery(ids, geometry):
    for image_id in ids: 
        print image_id
        dl.raster.raster(
                        image_id,
                        bands=['swir2', 'swir1', 'nir', 'alpha'],
                        scales=[[0,4000], [0, 4000], [0, 4000], None],
                        data_type='Byte',
                        cutline=geometry,
                        save= True,
                        outfile_basename="temp/"+image_id)

In [12]:
def find_imagery(fire_name, date, coords, footprint=None):
    # Where name is a str, coords is an array of [lat,long], and footprint is an optional geometry
#     if footprint:
#         image_ids = search_products(date,footprint)
#         download_imagery(image_ids, footprint)
#     else:
    dl_tile = make_tile(coords)
    image_ids = search_products(date, dl_tile)
    download_imagery(image_ids, dl_tile)

Driver Code

In [ ]:
find_imagery('Mallard Fire', '2018-05-10',[34.81, -101.31])         
    

In [ ]:
fires = {
    'id:5790':{
        'name': 'Mallard Fire',
        'date_entered': '2018-05-10',
        'last_checked': '2018-05-15'
    },
    'id:5795':{
        'name': 'Pinery Fire',
        'date_entered': '2018-05-10',
        'last_checked': '2018-05-15'
    }
}